# Data Sharing with Encryption - Delentture side

## Recommended 

Create conda enviroment to avoid conflicts with packages.

## Data Analysis

In [1]:
import pandas as pd
import os

pd.options.display.max_colwidth = 200

# get current directory
path = os.getcwd()
parent = os.path.dirname(path)

DIR_DATA = parent+ '/data/'

In [2]:
df_columns = pd.read_csv(DIR_DATA+'columns_description.csv')
df_loans = pd.read_csv(DIR_DATA+'infringement_dataset.csv')

In [3]:
df_columns.head()

,New name,Description
0,address,Client's address
1,age,Client's age in days at the time of application
2,annual_income,Income of the client
3,appendix_a,Did client provide document 2
4,appendix_b,Did client provide document 3


In [4]:
df_loans.head()

,loan_id,infringed,contract_type,gender,has_own_car,has_own_realty,num_children,annual_income,credit_amount,credit_annuity,...,SK_ID_CURR,avg_days_decision,past_avg_amount_annuity,past_avg_amt_application,past_avg_amt_credit,past_loans_approved,past_loans_refused,past_loans_canceled,past_loans_unused,past_loans_total
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,100002.0,606.000000,9251.775,179055.00,179055.00,1.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,100003.0,1305.000000,56553.990,435436.50,484191.00,3.0,0.0,0.0,0.0,3.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,100004.0,815.000000,5357.250,24282.00,20106.00,1.0,0.0,0.0,0.0,1.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,100006.0,272.444444,23651.175,272203.26,291695.50,5.0,3.0,1.0,0.0,9.0
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,100007.0,1222.833333,12278.805,150530.25,166638.75,6.0,0.0,0.0,0.0,6.0


In [5]:
df_loans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 69 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   loan_id                   307511 non-null  int64  
 1   infringed                 307511 non-null  int64  
 2   contract_type             307511 non-null  object 
 3   gender                    307511 non-null  object 
 4   has_own_car               307511 non-null  object 
 5   has_own_realty            307511 non-null  object 
 6   num_children              307511 non-null  int64  
 7   annual_income             307511 non-null  float64
 8   credit_amount             307511 non-null  float64
 9   credit_annuity            307499 non-null  float64
 10  goods_valuation           307233 non-null  float64
 11  income_type               307511 non-null  object 
 12  education                 307511 non-null  object 
 13  family_status             307511 non-null  o

### Data Pre-processing

In [6]:
def check_nans(df,cols = None):
    if cols is None:
        cols = df.columns
    for col in cols:
        print(f'{col} - {df[col].isnull().sum()}')

def check_unique(df, cols = None):
    if cols is None:
        cols = df.columns
    for col in cols:
        print(f'{col} - {df[col].unique()}\n')  

cols_drop =['first_name','last_name','address']

df_loans_processed = df_loans.copy()
df_loans_processed.shape

(307511, 69)

#### Cleaning

In [7]:
cols_aux = ['region_rating','mobilephone_reachable','SK_ID_CURR','car_age','family_status','gender']
cond_regex = df_columns['New name'].isin(cols_aux)

df_columns[cond_regex]

,New name,Description
23,car_age,Age of client's car
29,family_status,Family status of the client
31,gender,Gender of the client
40,mobilephone_reachable,"Was mobile phone reachable (1=YES, 0=NO)"
63,region_rating,"Our rating of the region where client lives (1,2,3)"


In [8]:
df_loans_processed[cols_aux].head()

,region_rating,mobilephone_reachable,SK_ID_CURR,car_age,family_status,gender
0,2,1,100002.0,NaN,Single / not married,M
1,1,1,100003.0,NaN,Married,F
2,2,1,100004.0,26.0,Single / not married,M
3,2,1,100006.0,NaN,Civil marriage,F
4,2,1,100007.0,NaN,Single / not married,M


In [9]:
check_nans(df_loans_processed,cols_aux)
cols_drop += ['SK_ID_CURR','car_age']


region_rating - 0
mobilephone_reachable - 0
SK_ID_CURR - 16454
car_age - 202929
family_status - 0
gender - 0


In [10]:
cols_unique = ['family_status','gender','region_rating','mobilephone_reachable']
check_unique(df_loans_processed,cols_unique)

family_status - ['Single / not married' 'Married' 'Civil marriage' 'Widow' 'Separated'
 'Unknown']

gender - ['M' 'F' 'XNA']

region_rating - [2 1 3]

mobilephone_reachable - [1 0]



In [11]:
df_loans_processed = df_loans_processed[df_loans_processed.gender != 'XNA']
check_unique(df_loans_processed,['gender'])

gender - ['M' 'F']



- Labels starting with **'provided_'**

In [12]:
regex_provided = 'provided_.*'
cond_regex = df_columns['New name'].str.match(regex_provided)
cols_aux = df_columns[cond_regex]['New name'].tolist()
df_columns[cond_regex]

,New name,Description
59,provided_email,"Did client provide email (1=YES, 0=NO)"
60,provided_homephone,"Did client provide home phone (1=YES, 0=NO)"
61,provided_mobilephone,"Did client provide mobile phone (1=YES, 0=NO)"
62,provided_workphone,"Did client provide work phone (1=YES, 0=NO)"


In [13]:
check_unique(df_loans_processed,cols_aux)

provided_email - [0 1]

provided_homephone - [0 1]

provided_mobilephone - [1 0]

provided_workphone - [1 0]



- Labels starting with **_type**

In [14]:
regex_type = '.*_type'
cond_regex = df_columns['New name'].str.match(regex_type)
cols_aux = df_columns[cond_regex]['New name'].tolist()
df_columns[cond_regex]

,New name,Description
24,contract_type,Identification if loan is cash or revolving
35,housing_type,"What is the housing situation of the client (renting, living with parents, ...)"
36,income_type,"Clients income type (businessman, working, maternity leave,…)"
49,occupation_type,What kind of occupation does the client have
50,organization_type,Type of organization where client works


In [15]:
check_unique(df_loans_processed,cols_aux)
check_nans(df_loans_processed,cols_aux)

contract_type - ['Cash loans' 'Revolving loans']

housing_type - ['House / apartment' 'Rented apartment' 'With parents'
 'Municipal apartment' 'Office apartment' 'Co-op apartment']

income_type - ['Working' 'State servant' 'Commercial associate' 'Pensioner' 'Unemployed'
 'Student' 'Businessman' 'Maternity leave']

occupation_type - ['Laborers' 'Core staff' 'Accountants' 'Managers' nan 'Drivers'
 'Sales staff' 'Cleaning staff' 'Cooking staff' 'Private service staff'
 'Medicine staff' 'Security staff' 'High skill tech staff'
 'Waiters/barmen staff' 'Low-skill Laborers' 'Realty agents' 'Secretaries'
 'IT staff' 'HR staff']

organization_type - ['Business Entity Type 3' 'School' 'Government' 'Religion' 'Other' 'XNA'
 'Electricity' 'Medicine' 'Business Entity Type 2' 'Self-employed'
 'Transport: type 2' 'Construction' 'Housing' 'Kindergarten'
 'Trade: type 7' 'Industry: type 11' 'Military' 'Services'
 'Security Ministries' 'Transport: type 4' 'Industry: type 1' 'Emergency'
 'Security' 'Trad

In [16]:
df_loans_processed = df_loans_processed[df_loans_processed['organization_type'] != 'XNA']
check_unique(df_loans_processed,['organization_type'])

organization_type - ['Business Entity Type 3' 'School' 'Government' 'Religion' 'Other'
 'Electricity' 'Medicine' 'Business Entity Type 2' 'Self-employed'
 'Transport: type 2' 'Construction' 'Housing' 'Kindergarten'
 'Trade: type 7' 'Industry: type 11' 'Military' 'Services'
 'Security Ministries' 'Transport: type 4' 'Industry: type 1' 'Emergency'
 'Security' 'Trade: type 2' 'University' 'Transport: type 3' 'Police'
 'Business Entity Type 1' 'Postal' 'Industry: type 4' 'Agriculture'
 'Restaurant' 'Culture' 'Hotel' 'Industry: type 7' 'Trade: type 3'
 'Industry: type 3' 'Bank' 'Industry: type 9' 'Insurance' 'Trade: type 6'
 'Industry: type 2' 'Transport: type 1' 'Industry: type 12' 'Mobile'
 'Trade: type 1' 'Industry: type 5' 'Industry: type 10' 'Legal Services'
 'Advertising' 'Trade: type 5' 'Cleaning' 'Industry: type 13'
 'Trade: type 4' 'Telecom' 'Industry: type 8' 'Realtor' 'Industry: type 6']



- Labels starting with **has**

In [17]:
regex_has = 'has.*'
df_columns[df_columns['New name'].str.match(regex_has)]

,New name,Description
33,has_own_car,Flag if the client owns a car
34,has_own_realty,Flag if client owns a house or flat


In [18]:
df_has = df_loans_processed.filter(regex= (regex_has))
check_unique(df_has)

has_own_car - ['N' 'Y']

has_own_realty - ['Y' 'N']



- Labels starting with **score**

In [19]:
regex_score = 'score.*'
df_columns[df_columns['New name'].str.match(regex_score)]

,New name,Description
64,score_ext_1,Normalized score from external data source
65,score_ext_2,Normalized score from external data source
66,score_ext_3,Normalized score from external data source


In [20]:
df_score = df_loans_processed.filter(regex= (regex_score))
df_score.head()

,score_ext_1,score_ext_2,score_ext_3
0,0.083037,0.262949,0.139376
1,0.311267,0.622246,NaN
2,NaN,0.555912,0.729567
3,NaN,0.650442,NaN
4,NaN,0.322738,NaN


In [21]:
check_nans(df_score)
cols_drop += df_score.columns.tolist()

score_ext_1 - 130859
score_ext_2 - 504
score_ext_3 - 49896


- Labels starting with **appendix**

In [22]:
regex_appendix = 'appendix.*'
df_columns[df_columns['New name'].str.match(regex_appendix)].head()

,New name,Description
3,appendix_a,Did client provide document 2
4,appendix_b,Did client provide document 3
5,appendix_c,Did client provide document 4
6,appendix_d,Did client provide document 5
7,appendix_e,Did client provide document 6


In [23]:
df_appendix = df_loans_processed.filter(regex= (regex_appendix))
check_unique(df_appendix)

appendix_a - [0 1]

appendix_b - [1 0]

appendix_c - [0 1]

appendix_d - [0 1]

appendix_e - [0 1]

appendix_f - [0 1]

appendix_g - [0 1]

appendix_h - [0 1]

appendix_i - [0 1]

appendix_j - [0 1]

appendix_k - [0 1]

appendix_l - [0 1]

appendix_m - [0 1]

appendix_n - [0 1]

appendix_o - [0 1]

appendix_p - [0 1]

appendix_q - [0 1]

appendix_r - [0 1]

appendix_s - [0 1]

appendix_t - [0 1]



- Labels starting with **num_**

In [24]:
regex_num = 'num_.*'
df_columns[df_columns['New name'].str.match(regex_num)]

,New name,Description
41,num_children,Number of children the client has
42,num_family_members,How many family members does client have
43,num_req_bureau_day,Number of enquiries to Credit Bureau about the client one day before application (excluding one hour before application)
44,num_req_bureau_hour,Number of enquiries to Credit Bureau about the client one hour before application
45,num_req_bureau_month,Number of enquiries to Credit Bureau about the client one month before application (excluding one week before application)
46,num_req_bureau_qrt,Number of enquiries to Credit Bureau about the client 3 month before application (excluding one month before application)
47,num_req_bureau_week,Number of enquiries to Credit Bureau about the client one week before application (excluding one day before application)
48,num_req_bureau_year,Number of enquiries to Credit Bureau about the client one day year (excluding last 3 months before application)


In [25]:
df_num = df_loans_processed.filter(regex= (regex_num))
check_unique(df_num)
check_nans(df_num)

cols_drop += ['num_req_bureau_hour']

num_children - [ 0  1  2  3  4  7  5  6  8  9 11 12 10 19 14]

num_family_members - [ 1.  2.  3.  4.  5.  6.  9.  7.  8. 10. 13. nan 14. 12. 20. 15. 16. 11.]

num_req_bureau_hour - [ 0. nan  1.  2.  3.  4.]

num_req_bureau_day - [ 0. nan  1.  3.  2.  4.  5.  6.  9.]

num_req_bureau_week - [ 0. nan  1.  3.  2.  4.  5.  6.  8.  7.]

num_req_bureau_month - [ 0. nan  1.  2.  6.  3.  5.  7.  9.  4. 11.  8. 16. 12. 14. 10. 13. 17.
 24. 19. 15. 23. 18. 27.]

num_req_bureau_qrt - [  0.  nan   1.   2.   4.   3.   8.   5.   6.   7. 261.  19.]

num_req_bureau_year - [ 1.  0. nan  2.  4.  5.  3.  8.  6.  9.  7. 10. 11. 16. 12. 23. 15. 14.
 22. 17. 13. 21. 19.]

num_children - 0
num_family_members - 2
num_req_bureau_hour - 33856
num_req_bureau_day - 33856
num_req_bureau_week - 33856
num_req_bureau_month - 33856
num_req_bureau_qrt - 33856
num_req_bureau_year - 33856


- Labels starting with **past_loans**

In [26]:
regex_past = 'past_loans.*'
df_columns[df_columns['New name'].str.match(regex_past)]

,New name,Description
54,past_loans_approved,Number of loans approved in the past for the client
55,past_loans_canceled,Number of loans canceled in the past for the client
56,past_loans_refused,Number of loans refused in the past for the client
57,past_loans_total,"Number of loans in the past for the client, regardless of the outcome"
58,past_loans_unused,Number of loans approved but not used in the past for the client


In [27]:
df_past_loans = df_loans.filter(regex= (regex_past))
check_unique(df_past_loans)
check_nans(df_past_loans)

past_loans_approved - [ 1.  3.  5.  6.  4.  7.  2. nan  8. 12.  0.  9. 10. 13. 15. 11. 17. 16.
 14. 18. 19. 23. 20. 24. 22. 27. 21.]

past_loans_refused - [ 0.  3.  1. nan  6.  2.  4.  5. 10. 11.  8.  7.  9. 12. 21. 13. 15. 14.
 24. 18. 17. 16. 20. 22. 19. 57. 25. 34. 28. 44. 51. 23. 27. 40. 31. 26.
 38. 58. 30. 52.]

past_loans_canceled - [ 0.  1. nan 10.  8.  3.  2.  5.  4. 12.  9.  7.  6. 11. 20. 16. 15. 13.
 17. 19. 18. 14. 32. 23. 30. 21. 25. 24. 31. 37. 58. 22. 26. 28. 36. 27.
 33. 29. 47. 39. 68. 43. 41. 35. 54. 34. 51.]

past_loans_unused - [ 0. nan  1.  3.  2.  4.  5.  6.  8.  7. 16. 10.]

past_loans_total - [ 1.  3.  9.  6.  5.  7.  4.  2. nan  8. 12. 22. 18. 10. 26. 11. 15. 14.
 13. 17. 16. 23. 24. 27. 20. 19. 21. 35. 29. 25. 31. 33. 50. 28. 36. 32.
 46. 41. 58. 37. 40. 43. 39. 30. 34. 60. 38. 42. 72. 44. 52. 63. 55. 67.
 61. 68. 59. 73. 49. 47. 54. 51. 62. 64. 45. 53.]

past_loans_approved - 16454
past_loans_refused - 16454
past_loans_canceled - 16454
past_loans_unused - 16

In [28]:
cols_int = df_past_loans.columns.values.tolist() + df_num.columns.values.tolist() 
cols_clean = cols_int
df_loans_processed.dropna(subset=cols_clean, inplace=True)
print(f'Rows of raw data: {df_loans.shape[0]}\nRows of cleaned data: {df_loans_processed.shape[0]}')

Rows of raw data: 307511
Rows of cleaned data: 206416


In [29]:
df_loans_processed = df_loans_processed.drop(columns = cols_drop)
df_loans_processed.shape[1]

60

In [30]:
max_nans = 3
df_loans_processed = df_loans_processed[df_loans_processed.isnull().sum(axis=1) < max_nans]
print(f'Rows: {df_loans_processed.shape[0]}')

Rows: 206416


#### Data Type Transformation

In [31]:
df_loans_processed = df_loans_processed.astype({col:'int64' for col in cols_int})
df_loans_processed[cols_int]

KeyError: "Only a column name can be used for the key in a dtype mappings argument. 'num_req_bureau_hour' not found in columns."

#### State of the data after processed

In [ ]:
df_loans_processed.info()
df_loans_processed.to_csv(DIR_DATA+'infringment_processed.csv')